In [1]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
from matplotlib import cm
import lightgbm as lgb
from lightgbm import LGBMClassifier

/Users/hanjiawei/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
data = pd.read_csv('Police_Department_Incidents_-_Previous_Year__2016_.csv', parse_dates=['Date'])

In [26]:
data.head(3)

NameError: name 'data' is not defined

In [5]:
def feature_engineering(data):
    data['Date'] = pd.to_datetime(data['Date'].dt.date)
    data['n_days'] = (
        data['Date'] - data['Date'].min()).apply(lambda x: x.days)
    data['Day'] = data['Date'].dt.day
    data['DayOfWeek'] = data['Date'].dt.weekday
    data['Month'] = data['Date'].dt.month
    data['Year'] = data['Date'].dt.year
    data['Block'] = data['Address'].str.contains('block', case=False)
        
    return data

In [6]:
data2 = feature_engineering(data)

In [7]:
data2.head(3)

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,n_days,Day,Month,Year,Block
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,4,2016-01-29,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120,28,29,1,2016,True
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",4,2016-01-29,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168,28,29,1,2016,True
2,141059263,WARRANTS,WARRANT ARREST,0,2016-04-25,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010,115,25,4,2016,False


In [8]:
data2.drop(columns=['Descript','Resolution','IncidntNum','Date','Address'], inplace=True)

In [9]:
data2.head(3)

,Category,DayOfWeek,Time,PdDistrict,X,Y,Location,PdId,n_days,Day,Month,Year,Block
0,WEAPON LAWS,4,11:00,SOUTHERN,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120,28,29,1,2016,True
1,WEAPON LAWS,4,11:00,SOUTHERN,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168,28,29,1,2016,True
2,WARRANTS,0,14:59,BAYVIEW,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010,115,25,4,2016,False


In [10]:
data2.drop(columns=['Location','PdId'], inplace=True)

In [11]:
data2['Time']

0         11:00
1         11:00
2         14:59
3         23:50
4         00:30
          ...  
150495    21:01
150496    08:00
150497    20:00
150498    10:00
150499    14:00
Name: Time, Length: 150500, dtype: object

In [12]:
def get_hour(DATE):
    hour = DATE[0:2]
    return hour
data2['hour']=data['Time'].apply(get_hour)

In [13]:
data2.head(3)

,Category,DayOfWeek,Time,PdDistrict,X,Y,n_days,Day,Month,Year,Block,hour
0,WEAPON LAWS,4,11:00,SOUTHERN,-122.403405,37.775421,28,29,1,2016,True,11
1,WEAPON LAWS,4,11:00,SOUTHERN,-122.403405,37.775421,28,29,1,2016,True,11
2,WARRANTS,0,14:59,BAYVIEW,-122.388856,37.729981,115,25,4,2016,False,14


In [14]:
def get_min(DATE):
    min = DATE[3:5]
    return min
data2['min']=data['Time'].apply(get_min)

In [15]:
data2

,Category,DayOfWeek,Time,PdDistrict,X,Y,n_days,Day,Month,Year,Block,hour,min
0,WEAPON LAWS,4,11:00,SOUTHERN,-122.403405,37.775421,28,29,1,2016,True,11,00
1,WEAPON LAWS,4,11:00,SOUTHERN,-122.403405,37.775421,28,29,1,2016,True,11,00
2,WARRANTS,0,14:59,BAYVIEW,-122.388856,37.729981,115,25,4,2016,False,14,59
3,NON-CRIMINAL,1,23:50,TENDERLOIN,-122.412971,37.785788,4,5,1,2016,False,23,50
4,NON-CRIMINAL,4,00:30,MISSION,-122.419672,37.765050,0,1,1,2016,False,00,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150495,ASSAULT,4,21:01,PARK,-122.453982,37.771428,364,30,12,2016,False,21,01
150496,NON-CRIMINAL,4,08:00,CENTRAL,-122.401857,37.796626,364,30,12,2016,False,08,00
150497,LARCENY/THEFT,3,20:00,CENTRAL,-122.412269,37.790673,363,29,12,2016,False,20,00
150498,LARCENY/THEFT,4,10:00,CENTRAL,-122.406659,37.788275,364,30,12,2016,True,10,00


In [16]:
data2.drop(columns=['Time','Year'], inplace=True)

In [2]:
import numpy as np

In [18]:
data2.drop_duplicates(inplace=True)
data2.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)

In [19]:
data2['X'].isnull().value_counts()

False    139120
Name: X, dtype: int64

In [20]:
data2['Category'].value_counts()

LARCENY/THEFT                  39695
NON-CRIMINAL                   17386
OTHER OFFENSES                 17070
ASSAULT                        11248
VANDALISM                       8421
VEHICLE THEFT                   5932
SUSPICIOUS OCC                  5687
BURGLARY                        5667
WARRANTS                        5468
ROBBERY                         3206
DRUG/NARCOTIC                   2971
MISSING PERSON                  2749
FRAUD                           2542
SECONDARY CODES                 1835
TRESPASS                        1803
WEAPON LAWS                     1300
STOLEN PROPERTY                  866
SEX OFFENSES, FORCIBLE           798
RECOVERED VEHICLE                730
DISORDERLY CONDUCT               656
FORGERY/COUNTERFEITING           576
DRUNKENNESS                      465
DRIVING UNDER THE INFLUENCE      363
PROSTITUTION                     323
ARSON                            273
KIDNAPPING                       250
EMBEZZLEMENT                     165
L

In [21]:
data2.to_csv('crime2016.csv')

In [3]:
data3=pd.read_csv('new_crime2.csv')

In [4]:
data3['Police_Districts'].isnull().value_counts()
data3.dropna(subset=['Police_Districts'],inplace=True)

In [5]:
data3['Police_Districts'].isnull().value_counts()

False    110463
Name: Police_Districts, dtype: int64

In [6]:
le1 = LabelEncoder()
data3['Location_Block'] = le1.fit_transform(data3['Location_Block'])

In [7]:
data3['Location_Block'].value_counts()

15423    687
145      515
13811    391
3497     360
3008     341
        ... 
17513      1
19560      1
13415      1
5219       1
0          1
Name: Location_Block, Length: 25262, dtype: int64

In [8]:
data3['Police_Districts'].value_counts()

15.0    9332
17.0    9114
11.0    8361
16.0    7760
14.0    6352
9.0     6196
18.0    6180
20.0    6085
2.0     5967
10.0    5204
5.0     5178
21.0    4968
3.0     4793
8.0     4718
19.0    4307
12.0    4242
13.0    3355
7.0     2787
1.0     2198
6.0     2028
4.0     1138
22.0     200
Name: Police_Districts, dtype: int64

In [9]:
le2 = LabelEncoder()
data3['CAT'] = le2.fit_transform(data3['CAT'])
data3['CAT'].value_counts()

2    45801
1    41134
0    12278
3    11250
Name: CAT, dtype: int64

In [10]:
data3.drop(columns=['Unnamed: 0'], inplace=True)

In [11]:
data3

,Location_Block,Police_Districts,Lon,Lat,day_time,n_days,Day,DayOfWeek,Month,CAT
0,20756,13.0,-75.166412,39.940070,12.83,114,25,5,4,2
1,3523,16.0,-75.160229,39.976134,18.30,278,6,1,10,2
2,3525,16.0,-75.163073,39.977872,0.82,281,9,4,10,2
3,18386,2.0,-75.075602,40.041270,8.65,333,30,0,11,2
4,9006,16.0,-75.178874,39.975724,12.48,343,10,3,12,2
...,...,...,...,...,...,...,...,...,...,...
110604,10720,18.0,-75.142924,39.999500,7.58,66,8,6,3,3
110605,13966,14.0,-75.213679,39.955537,2.08,304,1,6,11,3
110606,11200,21.0,-75.156581,40.003890,5.90,66,8,6,3,3
110607,547,5.0,-75.155966,39.954813,5.82,66,8,6,3,2


In [14]:
data3.to_csv('new_crime_forest2.csv')

In [5]:
data3.dropna(subset=['PdDistrict'],inplace=True)

In [6]:
data3['PdDistrict'].isnull().value_counts()

False    121669
Name: PdDistrict, dtype: int64

In [7]:
data3['CAT'].isnull().value_counts()

False    121669
Name: CAT, dtype: int64

In [8]:
le1 = LabelEncoder()
data3['PdDistrict'] = le1.fit_transform(data3['PdDistrict'])

In [9]:
data3['PdDistrict'].value_counts()

7    23125
4    16920
3    15278
1    15160
0    11120
2     9044
8     9016
9     7641
6     7580
5     6785
Name: PdDistrict, dtype: int64

In [10]:
data3

,Unnamed: 0,DayOfWeek,PdDistrict,X,Y,n_days,Day,Month,Block,hour,min,CAT
0,2,1,9,-122.412971,37.785788,4,5,1,False,23,50,OTHER
1,3,4,3,-122.419672,37.765050,0,1,1,False,0,30,OTHER
2,4,4,4,-122.426077,37.788019,0,1,1,True,21,35,VIOLENCE
3,5,5,7,-122.405721,37.780879,1,2,1,False,0,4,OTHER
4,6,5,9,-122.411778,37.783981,1,2,1,True,1,2,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...
121665,139115,4,5,-122.453982,37.771428,364,30,12,False,21,1,VIOLENCE
121666,139116,4,1,-122.401857,37.796626,364,30,12,False,8,0,OTHER
121667,139117,3,1,-122.412269,37.790673,363,29,12,False,20,0,THEFT
121668,139118,4,1,-122.406659,37.788275,364,30,12,True,10,0,THEFT


In [11]:
le2 = LabelEncoder()
data3['CAT'] = le2.fit_transform(data3['CAT'])


In [12]:
data3.drop(columns=['Unnamed: 0'], inplace=True)

In [13]:
data3['CAT'].value_counts()

2    55365
1    34456
3    27199
0     4649
Name: CAT, dtype: int64

In [14]:
data3.to_csv('new_crime_forest2.csv')

In [12]:
y=data3.pop('CAT')

In [13]:
y

0         2
1         2
2         2
3         2
4         2
         ..
110604    3
110605    3
110606    3
110607    2
110608    2
Name: CAT, Length: 110463, dtype: int64

In [15]:
data3.drop(columns=['Day','DayOfWeek','Month'], inplace=True)
data3

,Location_Block,Police_Districts,Lon,Lat,day_time,n_days
0,20756,13.0,-75.166412,39.940070,12.83,114
1,3523,16.0,-75.160229,39.976134,18.30,278
2,3525,16.0,-75.163073,39.977872,0.82,281
3,18386,2.0,-75.075602,40.041270,8.65,333
4,9006,16.0,-75.178874,39.975724,12.48,343
...,...,...,...,...,...,...
110604,10720,18.0,-75.142924,39.999500,7.58,66
110605,13966,14.0,-75.213679,39.955537,2.08,304
110606,11200,21.0,-75.156581,40.003890,5.90,66
110607,547,5.0,-75.155966,39.954813,5.82,66


In [16]:
X_train,X_test,y_train,y_test= train_test_split(data3,y,test_size=0.1,random_state=0)

In [17]:
import eli5
from eli5.sklearn import PermutationImportance

/Users/hanjiawei/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/hanjiawei/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [18]:
model =LGBMClassifier(objective='multiclass', num_class=39).fit(X_train, y_train)

perm = PermutationImportance(model).fit(X_test, y_test)
eli5.show_weights(perm, feature_names=X_test.columns.tolist())

Weight,Feature
0.0944 ± 0.0042,day_time
0.0835 ± 0.0044,Lat
0.0817 ± 0.0054,Lon
0.0285 ± 0.0021,Police_Districts
0.0259 ± 0.0044,Location_Block
0.0068 ± 0.0014,n_days


In [26]:
y_train

122834    2
103066    2
61892     2
35383     1
43074     3
         ..
41993     2
97639     1
95939     2
117953    2
43567     3
Name: CAT, Length: 111295, dtype: int64

In [35]:
import lightgbm as lgb
# Creating the model
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=['PdDistrict'])

params = {'boosting':'gbdt',
          'objective':'multiclass',
          'num_class':5,
          'max_delta_step':0.9,
          'min_data_in_leaf': 21,
          'learning_rate': 0.4,
          'max_bin': 465,
          'num_leaves': 41
         }

bst = lgb.train(params, train_data, 100)

predictions = bst.predict(X_test)


In [36]:
predictions

array([[9.59384797e-03, 2.50977271e-01, 5.40573186e-01, 1.98855696e-01,
        1.00954361e-15],
       [5.21736739e-02, 3.72267276e-01, 4.25122222e-01, 1.50436828e-01,
        1.04130964e-15],
       [2.78404478e-02, 5.57057491e-01, 2.21477359e-01, 1.93624702e-01,
        8.02376024e-16],
       ...,
       [2.80176354e-02, 5.04701108e-01, 2.03158000e-01, 2.64123257e-01,
        1.28474938e-15],
       [2.00031377e-02, 4.05487351e-01, 3.42831445e-01, 2.31678066e-01,
        9.94651043e-16],
       [3.71830882e-03, 1.21744769e-01, 8.12149199e-01, 6.23877232e-02,
        6.23306815e-16]])

In [37]:
len(predictions)

27824

In [38]:
len(X_test)

27824

In [45]:
# Submitting the results
import numpy as np
submission = pd.DataFrame(predictions,columns=le2.inverse_transform(np.linspace(0, 4, 5, dtype='int16')),index=X_test.index)
submission.to_csv('LGBM_final2.csv', index_label='Id')

ValueError: y contains previously unseen labels: [4]

In [46]:
np.linspace(0, 4, 5, dtype='int16')

array([0, 1, 2, 3, 4], dtype=int16)

In [43]:
le2

LabelEncoder()